#Narrative

I tried to add up the count for each narrative sets and remove the single word feature from the word matrix based on Count Vectorizer.

"Narrative To measure the usage of all five narratives in requests we use word count features that measure how often
a given requests mentions words from the previously defined narrative lexicons. We normalize these features by the total
number of words in the request to remove length effects. Here, we use median-thresholded binary variables for easier
interpretation but use decile-coded variants in the following prediction task."
--Thesis Page 5

However, the accuracy is not improved....

In [ ]:
#Load the data

import pandas, json
from sklearn.feature_extraction.text import *
import re
import numpy as np


# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report


json_data = json.load(open('pizza_request_dataset.json'))
df=pandas.io.json.json_normalize(json_data)
print(df.columns.values)
print(df.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(0)

from sklearn.cross_validation import train_test_split

train, dev = train_test_split(df, test_size = 0.3)

train_labels=train["requester_received_pizza"]
train_data = train
del train_data["requester_received_pizza"]

dev_labels=dev["requester_received_pizza"]
dev_data = dev
del dev_data["requester_received_pizza"]

print ('training label shape:', train_labels.shape)
print ('test shape:', train_data.shape)
print ('test label shape:', dev_labels.shape)
print ('test shape:', dev_data.shape)

Build count Vectorizer and test the accuracy scores

In [ ]:
def first_preprocessor(s):
    #convert to lowercase
    s=s.lower()
    s=re.sub("[,.!?:;/~*]"," ",s)
#     #remove duplicated 0s and 1s
    s=re.sub("[0-9]*","",s)
#     #remove number longer than 5 digit
    s=re.sub("[0-9]{5,}","",s)
#     #remove word longer than 20 digit
#     s=re.sub("[0-9a-z]{20,}","",s)
#     #remove stem end with 'ly'
    s=re.sub("ly\s"," ",s)
#     s=re.sub("ly\Z"," ",s)
#     #remove plural form
    s=re.sub("s\s"," ",s)
    s=re.sub("s\Z"," ",s)
#     #remove _ as the end of word
    s=re.sub("[_]+"," ",s)
#     #remove _ as start of the word
    s=re.sub("\s[_]+"," ",s)
#     #remove stem end with 'ful'
#     s=re.sub("ful\s"," ",s)
#     s=re.sub("ful\Z"," ",s)
#     #remove stem end with 'able'
#     s=re.sub("able\s"," ",s)
#     s=re.sub("able\Z"," ",s)
#     #remove stem end with 'ness'
    s=re.sub("ness\s"," ",s)
    s=re.sub("ing\s"," ",s)
#     s=re.sub("ing\Z"," ",s)
#     #remove stop words
#     s=re.sub(" about "," ",s)
    #remove words that are too short
    s=re.sub("\s[0-9a-z]{1,2}\s"," ",s)
    s=re.sub("\s[0-9a-z]{1,2}\Z"," ",s)

    return s


def empty_preprocessor(s):
    return s

vectorizer_process = CountVectorizer(preprocessor =empty_preprocessor,analyzer='word',stop_words='english' )
word_matrix_process= vectorizer_process.fit_transform(train_data['request_text'])
dev_matrix_process = vectorizer_process.transform(dev_data['request_text'])


model_LG = LogisticRegression(penalty ='l2',C=12)
model_LG.fit(word_matrix_process, train_labels)


f1_score=metrics.f1_score(dev_labels,model_LG.predict(dev_matrix_process),average='binary')
print(f1_score)
print(metrics.precision_score(dev_labels,model_LG.predict(dev_matrix_process)))
print(metrics.recall_score(dev_labels,model_LG.predict(dev_matrix_process)))

In [ ]:
from scipy import sparse
from scipy.sparse import csr_matrix 
from scipy.sparse import coo_matrix, hstack

def combine_features(feature_list,matrix,vectorizer):
    a=csr_matrix((matrix.shape[0],1), dtype=np.int8)
    #create count matric for each word in the list
    for word in feature_list:
        if vectorizer.vocabulary_.get(word)!=None:
            b=matrix[:,vectorizer_process.vocabulary_.get(word)]
            a= hstack([a,b],format= 'csr')
            
    #get the total count 
    a_sum= csr_matrix.sum(a,axis=1)
    #convert to the ndarray and remove the each single word feature
    del_ls=[]
    for word in feature_list:
        if vectorizer.vocabulary_.get(word)!=None:
            del_ls.append(vectorizer_process.vocabulary_.get(word))
    print(del_ls)
    
    matrix=matrix.toarray()
    matrix=np.delete(matrix,del_ls,axis=1)
    
    return (hstack([sparse.csr_matrix(matrix),a_sum],format= 'csr'))


desire = ['party','birthday','boyfriend','girlfriend','date','drinks','drunk','wasted','invite','invited','celebrate',
          'celebrating','game','games','movie','beer','crave','craving']
family = ['husband','wife','family','parent','parents','mother','father','mom','mum','son','dad','daughter']
job =    ['job','unemployment','employment','hire','hired','fired','interview','work','paycheck']
money=   ['money','bill','bills','rent','bank','account','paycheck','due','broke','bills','deposit','cash','dollar','dollars',
          'bucks','paid','payed','buy','check','spent','financial',
          'poor','loan','credit','budget','day','now','time','week','until','last','month','tonight','today','next','night',
          'when','tomorrow','first','after','while','before','long','hour','Friday','ago','still','due','past','soon','current',
          'years','never','till','yesterday','morning','evening']

for i in [desire,family,job,money]:
    word_matrix_process=combine_features(i,word_matrix_process,vectorizer_process)

print(word_matrix_process.shape)

for i in [desire,family,job,money]:
    dev_matrix_process=combine_features(i,dev_matrix_process,vectorizer_process)

print(dev_matrix_process.shape)
